# pymysql

-   파이썬에서 MySQL, MariaDB DBMS와 연동하는 다양한 함수를 제공하는 모듈
-   Python [DB API 2.0](http://www.python.org/dev/peps/pep-0249) 표준을 따른다.
-   https://github.com/PyMySQL/PyMySQL/
-   https://pymysql.readthedocs.io/en/latest/


# 설치

-   조건
    -   python version 3.6 이상
    -   mysql version 5.6 이상
-   설치
    -   `pip install pymysql`
    -   `conda install -c conda-forge pymysql`


# 기본 작성 절차

1. Database 연결
    ```python
       connection =  pymysql.connect(host:str="DBMS 서버 ip",
                                     port:int=port번호,
                                     user:str="계정명",
                                     password:str="비밀번호",
                                     db:str="연결할데이터베이스이름")
    ```
    - port 번호 기본값: 3306
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
2. Connection을 이용해 Cursor 생성
    - Cursor: 연결된 Database에 sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    ```python
        cursor = connection.cursor()
    ```
    - ContextManager 타입으로 with 구문을 이용해 작성하면 close() 작업을 자동으로 처리한다.
3. Cusror를 이용해 SQL문 실행(DB Server로 전송)
    ```python
        cursor.execute("sql문")
    ```
4. Select 결과 조회
    - select 문을 실행한 경우 cursor를 의 fetch 메소드들을 이용해 select 결과를 조회한다.
    ```python
    result = cursor.fetchall()
    ```
5. 연결 닫기
    - cursor, connection 연결을 닫는다.
    - with문을 이용할 수 있다.
    ```python
    cursor.close()
    connection.close()
    ```


# 예제

## 테이블 생성


In [2]:
create_sql = """
create table customer(
  id  int  auto_increment  primary key,
  name  varchar(20) not null,
  email  varchar(50) not null unique, 
  tall   double,
  birthday  date,
  created_at  datetime  not null
)
"""   
# 파이썬 내에서 sql 문 마지막에 `;` 은 붙이지 않는다.

In [3]:
import pymysql
try:
    conn = None
    
    # 1. Database와 연결.
    conn = pymysql.connect(
        host="127.0.0.1",    # DBMS 의 ip(host) : str
        port=3306,           # DBMS의 port 번호: int
        user="scott",        # username: str
        password="tiger",    # password: str
        db="mydb"            #  연결할 Database이름: str
    )  # 연결 성공하면 연결된 DB와 관련 작업할 수있는 기능을 제공하는 Connection객체를 반환
    
    # 2. Connection을 사용해서 Cursor 객체 생성
    #    Cursor: sql 처리를 하는 기능을 제공.( sql 전송하고 처리결과를 받을 때까지를 관리)
    cursor = conn.cursor()
        
    # 3. SQL 문 전송
    cursor.execute("drop table if exists customer")
    cursor.execute(create_sql) 
    
finally:
    # 4. 연결닫기(끊기)
    if conn:
        cursor.close() # 4-1. cursor 연결 닫기
        conn.close()  # 4-2. connection 연결 닫기

## 파이썬 타입과 연결된 sql 데이터타입 
- str - 문자열타입(char, varchar, text,...)
- int - 정수(tiny int , int, ....)
- float - 실수(float, double)
- decimal.Decimal - 실수(decimal)
- datetime.date - date
- datetime.time - time
- datetime.datetime - datetime, timestamp

> ### datetime 모듈
> - 파이썬에서 날짜, 시간을 다루는 모듈
> - 날짜 type(class): date
> - 시간 type: time
> - 날짜시간 type: datetime

In [ ]:
from datetime import date, datetime, time
a = date(2000, 10, 20)                                    # 날짜 객체 생성 - 년, 월, 일
print(a.year, a.month, a.day)                             # data객체에서 년/월/일 조회
b = datetime(2000,  10, 20, 12, 23)                       # 날짜시간 객체 생성 - 년, 월, 일, 시, 분, 초 지정. (초는 생략가능)
print(b.year, b.month, b.day, b.hour, b.minute, b.second) # 년/월/일/시/분/초 조회

print(datetime.now())  # 실행시점의 일시로 datetime객체 생성
print(date.today())    # 실행시점의 날짜로 date객체 생성

## DML

### insert


In [ ]:
sql = "insert into customer (name, email, tall, birthday, created_at) values('이순신', 'lee1@naver.com', 185.23, '2000-09-20', now())"

In [ ]:
# connection, cursor 생성 -> with 문으로 작성.
# manual commit 이 default. 
##  DML(insert/update/delete) 처리 후 commit을 실행해야한다.
with pymysql.connect(host="127.0.0.1", port=3306, user="scott", password="tiger", db="mydb") as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("처리 행수:", result)
        # commit 처리
        conn.commit()

### Parameterized Query

-   Parameterized Query
    -   SQL 문에서 컬럼 값이 들어가는 자리에 값대신 `%s` placeholder를 사용한뒤 execute()에서 placeholder에 넣을 값을 list나 tuple로 제공한다.
    -   동일한 쿼리문을 값을 바꿔가면서 여러번 실행할 때 유용하다.


In [4]:
import pymysql

# insert할 값 입력받기
name = input("이름:")
email = input("이메일주소:")

# SQL 실행
insert_sql = "insert into customer (name, email, tall, birthday, created_at) values (%s, %s, %s, %s, now())"

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(insert_sql, [name, email, 175.23, "2010-01-01"])
        conn.commit()
        print("처리 행수:", result)

이름: 가나다
이메일주소: abc@a.com


처리 행수: 1


#### Parameterized Query를 이용해 여러개 행 insert

- executemany() 사용
    - insert할 값들을 가진 리스트를 넣어 한번에 여러 행을 insert한다.


In [6]:
from datetime import datetime, date
datas = [
    ["김인영", "abc2@a.com", 165, date(2005, 1, 12), datetime.now()],
    ["오수철", "def2@a.com", 175, date(1995, 12, 20), datetime.now()],
    ["최유명", "ghi2@a.com", 183, date(1978, 10, 28), datetime.now()],
    ["김명수", "jkl@abc.com", 177, date(2000, 2, 12), datetime.now()],
    ["이지영", "mno@abc.com", 163, date(1995, 4, 21), datetime.now()],
    ["박명수", "pqr@abc.com", 185, date(2002, 7, 5), datetime.now()],
]

In [7]:
insert_sql = "insert into customer(name, email, tall, birthday, created_at) values(%s, %s, %s, %s, %s)"
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        cnt = cursor.executemany(insert_sql, datas)  # executemany() 결과 행수 반환. for문 대신 돌려주는것
        conn.commit()

print("insert된 총 행수:", cnt)

insert된 총 행수: 6


### update/delete

-   코딩 절차는 insert 와 동일


In [13]:
update_sql = "update customer set email=%s, tall=%s where id=%s"
tall = float(input("변경할 키: "))
cust_id = int(input("변경할 고객 ID: "))
email = str(input("이메일 : "))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(update_sql, [email, tall, cust_id])
        print("처리 행수: ", result)
        conn.commit()

변경할 키:  180
변경할 고객 ID:  1
이메일 :  aaaaa@a.com


처리 행수:  1


In [14]:
delete_sql = "delete from customer where tall > %s"

tall = float(input("삭제기준 키:"))

with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(delete_sql, [tall])
        # result = cursor.execute(delete_sql, (tall,))  # tuple로 작업할때는 값이 하나일 때 , 까먹지 않기
        print("처리 행수: ", result)
        conn.commit()

삭제기준 키: 180


처리 행수:  2


## select (DQL - Data Query Language)

-   조회결과 조회
    -   cursor.execute("select문") 실행 후 cursor의 결과 조회 메소드(fetch메소드)를 이용해 결과를 받는다.
-   fetch메소드
    -   **fetchall()**
        -   조회한 모든 행을을 반환
    -   **fetchmany(size=개수)**
        -   전체 조회한 행들 중 지정한 size개수 만큼 반환.
        -   연속적으로 실행하면 다음 size개수 만큼씩 반환한다.
        -   더 이상 조회한 결과가 없으면 빈 튜플을 반환한다.
    -   **fetchone()**
        -   조회결과 중 첫번째 행만 반환
        -   주로 pk 동등 조건으로 조회한 경우 사용


### fetchall()


In [19]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

for i in resultset :
    print(f"{i[1]}의 키는 {i[2]}")

조회행수: 5
가나다의 키는 180.0
김인영의 키는 165.0
오수철의 키는 175.0
김명수의 키는 177.0
이지영의 키는 163.0


In [ ]:
resultset

### 조회결과를 dictionary로 반환
- pymysql.cursors.DictCursor 사용
    - Connection 생성시 또는 Cursor 생성시 지정한다.
- key: 컬럼명, value: 컬럼값

In [34]:
sql = "select id, name, tall from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor(pymysql.cursors.DictCursor) as cursor:  
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset = cursor.fetchall()

조회행수: 5


In [35]:
for i in resultset :
    print(i)

{'id': 1, 'name': '가나다', 'tall': 180.0}
{'id': 2, 'name': '김인영', 'tall': 165.0}
{'id': 3, 'name': '오수철', 'tall': 175.0}
{'id': 5, 'name': '김명수', 'tall': 177.0}
{'id': 6, 'name': '이지영', 'tall': 163.0}


### fetchone()


In [36]:
sql = "select * from customer where id = %s" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql, [2])
        print("조회행수:", result)
        resultset = cursor.fetchone()

조회행수: 1


In [42]:
resultset
resultset[2]

'abc2@a.com'

### fetchmany()


In [43]:
sql = "select id, name, birthday from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        print("조회행수:", result)
        resultset1 = cursor.fetchmany(size=2)  # 처음 두개
        resultset2 = cursor.fetchmany(size=2)  # 다음 두개
        resultset3 = cursor.fetchmany(size=2)
        resultset4 = cursor.fetchmany(size=2)

조회행수: 5


In [44]:
print(resultset1)
print(resultset2)
print(resultset3)
print(resultset4)

((1, '가나다', datetime.date(2010, 1, 1)), (2, '김인영', datetime.date(2005, 1, 12)))
((3, '오수철', datetime.date(1995, 12, 20)), (5, '김명수', datetime.date(2000, 2, 12)))
((6, '이지영', datetime.date(1995, 4, 21)),)
()


### select문을 실행 한 cursor 는 iterable 타입

-   for in 문에 select query를 실행한 cursor를 사용하면 조회결과를 한 행씩 조회할 수 있다.


In [47]:
sql = "select * from customer" 
with pymysql.connect(host="127.0.0.1", port=3306, user='scott', password='tiger', db='mydb') as conn:
    with conn.cursor() as cursor:
        result = cursor.execute(sql)
        for id, name, email, tall, birthday, created_at in cursor:
            print(id, name, email, tall, birthday, created_at, sep=" , ")
        # for v in cursor :
            # print(v)

1 , 가나다 , aaaaa@a.com , 180.0 , 2010-01-01 , 2024-10-10 11:17:34
2 , 김인영 , abc2@a.com , 165.0 , 2005-01-12 , 2024-10-10 11:21:51
3 , 오수철 , def2@a.com , 175.0 , 1995-12-20 , 2024-10-10 11:21:51
5 , 김명수 , jkl@abc.com , 177.0 , 2000-02-12 , 2024-10-10 11:21:51
6 , 이지영 , mno@abc.com , 163.0 , 1995-04-21 , 2024-10-10 11:21:51


### database.py 모듈 활용해서 함수 실행

In [51]:
from database import HRDao
import configparser as parser

In [67]:
### configparser에서 mysql 정보 저장
props = parser.ConfigParser()
props.read('./config.ini')
mysql_config = props['MYSQL'] # 설정 정보 조회
print('host :', mysql_config['host'])

hr = HRDao(mysql_config['host'], 3306, mysql_config['user'], mysql_config['password'], mysql_config['db'])
hr.select_job()
all_emp = hr.select_all_emp()
for i in all_emp :
    print(i)

all_dept = hr.select_dept()
for i in all_dept :
    print(i)

host : 127.0.0.1
{'emp_id': 100, 'emp_name': 'Steven', 'dept_name': 'Executive', 'job_title': 'President', 'hire_date': datetime.date(2024, 10, 8), 'salary': '$51,000'}
{'emp_id': 101, 'emp_name': 'Neena', 'dept_name': 'Executive', 'job_title': 'Administration Vice President', 'hire_date': datetime.date(2024, 10, 8), 'salary': '$17,000'}
{'emp_id': 102, 'emp_name': 'Lex', 'dept_name': 'Executive', 'job_title': 'Administration Vice President', 'hire_date': datetime.date(2024, 10, 8), 'salary': '$17,000'}
{'emp_id': 103, 'emp_name': 'Alexander', 'dept_name': 'IT', 'job_title': 'Programmer', 'hire_date': datetime.date(2024, 10, 8), 'salary': '$27,000'}
{'emp_id': 104, 'emp_name': 'Bruce', 'dept_name': 'IT', 'job_title': 'Programmer', 'hire_date': datetime.date(2024, 10, 8), 'salary': '$18,000'}
{'emp_id': 105, 'emp_name': 'David', 'dept_name': 'IT', 'job_title': 'Programmer', 'hire_date': datetime.date(2024, 10, 8), 'salary': '$14,400'}
{'emp_id': 106, 'emp_name': 'Valli', 'dept_name': 'I